# Advanced Regression Challenge
This is a Thinkful assignment for Advanced Regreession. This assignment will be using the League of Legends Ranked games data and predict the result at the end of thee game, which either team#1 won or team#2 won. The data used for this assignment is from Kaggle. Link below. 

https://www.kaggle.com/datasnaek/league-of-legends

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
%matplotlib inline
sns.set_style('white')

In [0]:
from google.colab import files
uploaded = files.upload()

Saving games.csv to games.csv


In [0]:
df = pd.read_csv('games.csv')


In [0]:
df.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51490 entries, 0 to 51489
Data columns (total 61 columns):
gameId                51490 non-null int64
creationTime          51490 non-null int64
gameDuration          51490 non-null int64
seasonId              51490 non-null int64
winner                51490 non-null int64
firstBlood            51490 non-null int64
firstTower            51490 non-null int64
firstInhibitor        51490 non-null int64
firstBaron            51490 non-null int64
firstDragon           51490 non-null int64
firstRiftHerald       51490 non-null int64
t1_champ1id           51490 non-null int64
t1_champ1_sum1        51490 non-null int64
t1_champ1_sum2        51490 non-null int64
t1_champ2id           51490 non-null int64
t1_champ2_sum1        51490 non-null int64
t1_champ2_sum2        51490 non-null int64
t1_champ3id           51490 non-null int64
t1_champ3_sum1        51490 non-null int64
t1_champ3_sum2        51490 non-null int64
t1_champ4id           51490 non

Accoridng to the information of this data set, seems like this data set is fairly clean and all data points are numeric data already. It will save us lots of time to clean the data. According to the description of the data set, we will drop gameID, creation time, and season ID from our regression model. Actually we could have drop champion ID, summor spell ID, and ban ID as well, since they are champion ID and skill ID from Riot database. However, I will keep these columns here for the regression model because I want to see how these irrelevant data will effect our model. We will build another model that drop all these data and only keep inhibitor, dragon, baron, and rift herald data. 

In [0]:
# First, we create a data frame that exluded gameId, creationTime, and seasonId. 
df_all = df[df.columns[~df.columns.isin(['gameId','creationTime','seasonId'])]]

In [0]:
df_all.head()

,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,1949,1,2,1,1,1,1,2,8,12,...,5,0,0,1,1,114,67,43,16,51
1,1851,1,1,1,1,0,1,1,119,7,...,2,0,0,0,0,11,67,238,51,420
2,1493,1,2,1,1,1,2,0,18,4,...,2,0,0,1,0,157,238,121,57,28
3,1758,1,1,1,1,1,1,0,57,4,...,0,0,0,0,0,164,18,141,40,51
4,2094,1,2,1,1,1,1,0,19,4,...,3,0,0,1,0,86,11,201,122,18


### Complete Data

In [0]:
# First, we will create our training set and test set. 
X_all = df_all[df_all.columns[~df_all.columns.isin(['winner'])]]
Y_all = df_all['winner']

X_all_train, X_all_test, Y_all_train, Y_all_test = train_test_split(X_all, Y_all, test_size=0.3, train_size=0.7)

In [0]:
# We will fit our data set to Logistic Regression, Ridge Regression, and Lasso Regression. 
# Logistic Regression
lr = LogisticRegression()
fit = lr.fit(X_all_train, Y_all_train)

print('Coefficients')
print(fit.coef_)
print(fit.intercept_)

print('\n Percentage accuracy')
print(lr.score(X_all_test, Y_all_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Coefficients
[[ 1.78740940e-04  5.77158968e-02 -6.89016869e-01 -5.01894444e-01
  -1.84549102e-02  2.72397294e-01 -1.47211853e-01  4.05856209e-05
  -6.76483327e-03  3.54717026e-03 -2.15901168e-04  1.03921960e-02
   5.35999411e-03  5.85894189e-04  1.38232581e-02 -1.39529023e-03
  -2.51961447e-04  2.86370211e-03  1.61490096e-02  5.59570883e-04
   1.66808025e-02  6.67454286e-03 -1.17654378e+00  1.37449839e-01
  -4.97188396e-01  2.62477095e-02  5.11536701e-01  6.58228794e-04
   1.96896589e-04 -2.14374765e-04  1.84069779e-05  1.43011924e-04
  -5.77648737e-04 -3.15007971e-02 -2.04867589e-02 -3.87462886e-04
  -1.16477868e-02 -1.27463574e-02  1.07115922e-04 -6.97187238e-03
  -1.97537928e-02  2.96985401e-05 -8.60920150e-03 -8.68585214e-03
  -4.23186921e-05 -1.89652793e-02 -1.53859854e-02  1.37222827e+00
  -3.48962350e-01  3.93422142e-01 -8.85017156e-02 -3.29374277e-01
  -1.10428296e-05 -1.83928278e-04 -7.13434590e-05  1.35339059e-05
  -2.90346187e-04]]
[0.58915747]

 Percentage accuracy
0.960121

In [0]:
# Ridge Regression 
ridgeregr = linear_model.Ridge(alpha=100, fit_intercept=True)
ridgeregr.fit(X_all_train, Y_all_train)

print('Coefficients')
print(ridgeregr.coef_)
print(ridgeregr.intercept_)

print('\n Percentage accuracy')
print(ridgeregr.score(X_all_test, Y_all_test))

Coefficients
[ 3.50205815e-06  5.80794149e-03 -4.43109144e-02  2.03119607e-02
 -9.40310723e-03 -1.80566935e-04 -4.91718959e-03  1.00329687e-06
 -2.68358994e-04  2.52603571e-04  5.08491965e-07  3.03746250e-04
  2.56482688e-04  2.66712801e-05  8.28537561e-04  2.94591137e-04
 -1.40860159e-06 -1.24012486e-04  4.85622563e-04  2.17196999e-05
  1.31285867e-03  6.06011597e-04 -6.33328524e-02  2.87663776e-03
 -4.32559168e-02 -5.14486985e-03  3.04322724e-02  1.59535281e-05
  1.18306771e-05 -4.70378828e-06  1.84173498e-05  3.74769046e-06
 -1.87472798e-05 -1.17142878e-03 -6.62991964e-04 -2.37858437e-05
 -3.13632425e-04 -7.55922511e-04 -1.83317616e-05 -5.76820199e-04
 -6.57603172e-04 -1.00041634e-05 -5.89315314e-04 -5.72225270e-04
 -1.31979375e-05 -8.63883337e-04 -7.15748697e-04  6.69895419e-02
 -1.24476418e-02  5.28093131e-02  3.50537794e-03 -1.76747310e-02
  3.72927179e-06  4.66477061e-06  1.25221000e-05 -1.87202968e-05
 -1.32763825e-05]
1.5410123608651016

 Percentage accuracy
0.7896194410150078

In [0]:
# Lasso Regression
lassoregr = linear_model.Lasso(alpha=0.0001)
lassoregr.fit(X_all_train, Y_all_train)

# Since lasso regression is able to help us reduce our features by looking at the coefficients.
# Therefore, we would like to store our coefficients and feature names to a data frame and check them out later. 
coeff_name = pd.DataFrame({'coeff':list(zip(lassoregr.coef_))})
coeff_name['name'] = X_all_train.columns

print('Coefficients')
print(lassoregr.coef_)
print(lassoregr.intercept_)

print('\n Percentage accuracy')
print(lassoregr.score(X_all_test, Y_all_test))

Coefficients
[ 3.55877207e-06  5.46754311e-03 -4.44014691e-02  1.99153033e-02
 -9.19957910e-03  0.00000000e+00 -7.66605294e-05  1.01806497e-06
 -2.69944864e-04  2.43140999e-04  4.96851782e-07  2.87457247e-04
  2.41458288e-04  2.66683795e-05  8.13690357e-04  2.79393498e-04
 -1.36688268e-06 -1.24659455e-04  4.76903996e-04  2.17513312e-05
  1.29719490e-03  5.91318187e-04 -6.32968923e-02  2.72785004e-03
 -4.33660010e-02 -5.08514344e-03  2.55738744e-02  1.59475153e-05
  1.18267847e-05 -4.73144554e-06  1.83942392e-05  3.73981704e-06
 -1.87679309e-05 -1.15243975e-03 -6.44287324e-04 -2.38018051e-05
 -2.94493553e-04 -7.37806547e-04 -1.83426455e-05 -5.58321738e-04
 -6.39681037e-04 -1.00283333e-05 -5.69342880e-04 -5.54574454e-04
 -1.31922403e-05 -8.44898193e-04 -6.97839959e-04  6.70214785e-02
 -1.23050491e-02  5.27077378e-02  3.33817760e-03 -2.70110477e-02
  3.70620185e-06  4.65583604e-06  1.24947555e-05 -1.87084924e-05
 -1.32672017e-05]
1.5409049002846968

 Percentage accuracy
0.7896203476390213

In [0]:
print(coeff_name)

                         coeff                name
0     (3.558772069580713e-06,)        gameDuration
1      (0.005467543108733546,)          firstBlood
2      (-0.04440146913793679,)          firstTower
3      (0.019915303329525517,)      firstInhibitor
4     (-0.009199579100164858,)          firstBaron
5                       (0.0,)         firstDragon
6    (-7.666052939721166e-05,)     firstRiftHerald
7    (1.0180649695388418e-06,)         t1_champ1id
8    (-0.0002699448638797523,)      t1_champ1_sum1
9    (0.00024314099873816455,)      t1_champ1_sum2
10    (4.968517817632653e-07,)         t1_champ2id
11     (0.000287457247270172,)      t1_champ2_sum1
12   (0.00024145828822703054,)      t1_champ2_sum2
13   (2.6668379452230704e-05,)         t1_champ3id
14    (0.0008136903570730415,)      t1_champ3_sum1
15   (0.00027939349819016275,)      t1_champ3_sum2
16  (-1.3668826840632995e-06,)         t1_champ4id
17  (-0.00012465945490566162,)      t1_champ4_sum1
18   (0.00047690399592953777,) 

### Reduced Data

In [0]:
X_reduced = df[['gameDuration', 'firstBlood', 'firstTower', 'firstInhibitor',
                 'firstBaron', 'firstDragon','firstRiftHerald', 't1_towerKills',
                't1_inhibitorKills','t1_baronKills','t1_dragonKills',
                 't1_riftHeraldKills','t2_towerKills','t2_inhibitorKills',
                 't2_baronKills', 't2_dragonKills', 't2_riftHeraldKills']]

Y_reduced = df['winner']

X_reduced_train, X_reduced_test, Y_reduced_train, Y_reduced_test = train_test_split(X_reduced, Y_reduced, test_size=0.3, train_size=0.7)

In [0]:
# Logistic Regression
lr_reduced = LogisticRegression()
lr_reduced.fit(X_reduced_train, Y_reduced_train)

print('Coefficients')
print(lr_reduced.coef_)
print(lr_reduced.intercept_)

print('\n Percentage accuracy')
print(lr_reduced.score(X_reduced_test, Y_reduced_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Coefficients
[[ 2.33878035e-04  1.15617212e-01 -6.98659512e-01 -5.36409193e-01
   8.20685435e-03  2.34601801e-01 -1.08091645e-01 -1.18590802e+00
   1.62945715e-01 -5.47155965e-01  4.25807994e-02  5.06883310e-01
   1.36573985e+00 -2.78611472e-01  2.89369715e-01 -8.10960337e-02
  -3.07487477e-01]]
[-0.08818424]

 Percentage accuracy
0.9601864439697029


In [0]:
# Ridge Regression 
ridgeregr_reduced = linear_model.Ridge(alpha=10, fit_intercept=True)
ridgeregr_reduced.fit(X_reduced_train, Y_reduced_train)

print('Coefficients')
print(ridgeregr_reduced.coef_)
print(ridgeregr_reduced.intercept_)

print('\n Percentage accuracy')
print(ridgeregr_reduced.score(X_reduced_test, Y_reduced_test))

Coefficients
[ 4.14818385e-06  5.90254374e-03 -4.46593749e-02  2.34239876e-02
 -1.18067594e-02 -2.64005344e-04 -4.40006168e-03 -6.37573422e-02
  3.59464410e-03 -4.44498938e-02 -4.80462713e-03  3.21335901e-02
  6.65503981e-02 -1.02854149e-02  5.52255748e-02  3.22718171e-03
 -1.82668259e-02]
1.5181661711943617

 Percentage accuracy
0.7912107354242914


In [0]:
# Lasso Regression
lassoregr_reduced = linear_model.Lasso(alpha=0.0001)
lassoregr_reduced.fit(X_reduced_train, Y_reduced_train)

# Since lasso regression is able to help us reduce our features by looking at the coefficients.
# Therefore, we would like to store our coefficients and feature names to a data frame and check them out later. 
#coeff_name = pd.DataFrame({'coeff':list(zip(lassoregr.coef_))})
#coeff_name['name'] = X_all_train.columns

print('Coefficients')
print(lassoregr_reduced.coef_)
print(lassoregr_reduced.intercept_)

print('\n Percentage accuracy')
print(lassoregr_reduced.score(X_reduced_test, Y_reduced_test))

Coefficients
[ 4.15307980e-06  5.43650182e-03 -4.42003476e-02  2.26040970e-02
 -1.08623512e-02 -0.00000000e+00 -7.97841801e-05 -6.36648770e-02
  3.30200123e-03 -4.45845856e-02 -4.71591203e-03  2.73773284e-02
  6.65038697e-02 -9.75084075e-03  5.39109900e-02  3.08282051e-03
 -2.64691451e-02]
1.5181752942450775

 Percentage accuracy
0.7912146479777051


From above regression scores, we can clearly see that the logistic regression out-performed both Ridge regression and Lasso regression. This is because our model is looking for binary catagorial outcome, which means this is a classifier model instead of a regression model. Even though both Ridge regression and Lasso Regression model is able to used a classifier model, it does not mean that these two models will fit our data set th best and provide us the best result. 

Next, we will be looking at the actual Ridge classification model and Lasso classification model. For Lasso classification model, since SKLearn itself do not have a specific Lasso classification model, we will be using the Lasso Logistic regreession, which is setting our penalty to L1. 

In [0]:
# Ridge Classification
ridgecla_reduced = linear_model.RidgeClassifier(alpha=10, fit_intercept=True)
ridgecla_reduced.fit(X_reduced_train, Y_reduced_train)

print('Coefficients')
print(ridgecla_reduced.coef_)
print(ridgecla_reduced.intercept_)

print('\n Percentage accuracy')
print(ridgecla_reduced.score(X_reduced_test, Y_reduced_test))

Coefficients
[[ 8.29636770e-06  1.18050875e-02 -8.93187498e-02  4.68479752e-02
  -2.36135187e-02 -5.28010687e-04 -8.80012336e-03 -1.27514684e-01
   7.18928820e-03 -8.88997875e-02 -9.60925426e-03  6.42671802e-02
   1.33100796e-01 -2.05708299e-02  1.10451150e-01  6.45436342e-03
  -3.65336518e-02]]
[0.03633234]

 Percentage accuracy
0.9585032692432187


In [0]:
# Lasso Logistic Regression
llr_reduced = LogisticRegression(penalty='l1')
llr_reduced.fit(X_reduced_train, Y_reduced_train)

print('Coefficients')
print(llr_reduced.coef_)
print(llr_reduced.intercept_)

print('\n Percentage accuracy')
print(llr_reduced.score(X_reduced_test, Y_reduced_test))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Coefficients
[[ 2.40564135e-04  1.18620783e-01 -7.04751717e-01 -5.39693451e-01
   1.85654714e-02  2.38616696e-01 -1.24111686e-04 -1.18827397e+00
   1.59177697e-01 -5.52436842e-01  3.95363290e-02  4.01852551e-01
   1.36726339e+00 -2.72894843e-01  2.86738826e-01 -8.27741973e-02
  -5.23401097e-01]]
[-0.09668227]

 Percentage accuracy
0.960251181459183


From above Ridge Classification and Lasso Logistic Regression models, we can see that using the correct really help increase our model accuracy based on our data set.  Next, we will explore Gridsearch and try to find the best parameters for our hyper parameters. 

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
ridge_grid = {'alpha':np.arange(1,100,1)}

ridge_reduced_grid_search = GridSearchCV(ridgecla_reduced, ridge_grid, cv=10, scoring="balanced_accuracy")
ridge_reduced_grid_search.fit(X_reduced_train, Y_reduced_train)
print("Best Ridge-Reduced Classification alpha: ", ridge_reduced_grid_search.best_params_)


Best Ridge-Reduced Classification alpha:  {'alpha': 89}


In [0]:
print(ridge_reduced_grid_search.best_score_)

0.95947000125986


In [0]:
lasso_grid = {'alpha':np.arange(0.00001,0.001,0.0001)}

lasso_reduced_grid_search = GridSearchCV(lassoregr_reduced, lasso_grid, cv=10, scoring="r2")
lasso_reduced_grid_search.fit(X_reduced_train, Y_reduced_train)
print("Best Lasso Regression Reduced alpha: ", lasso_reduced_grid_search.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of

Best Lasso Regression Reduced alpha:  {'alpha': 0.00011}


This is a practice of how to use GridSearchCV to optimize our hyper-parameters. We can apply the same methon to other supervised learning models as well. 